<a href="https://colab.research.google.com/github/wagner-alves-AI/PLN/blob/master/Classificador_de_SPAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ETAPA DE OBTENÇÃO DOS DADOS

In [1]:
import requests #biblioteca HTTP
import zipfile  #biblioteca de compressão de arquivos
import io # biblioteca para lidar com entrada e saída

#URL  do arquivo TSV
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'

# Download do arquivos TSV
r = requests.get(url)

# Instanciando o compressor ZipFile
z = zipfile.ZipFile(io.BytesIO(r.content))

# Extração (descompressão) dos dados
z.extractall()

In [2]:
import pandas as pd

# Carregar o dataset de mensagens para um Dataframe
df = pd.read_table('/content/SMSSpamCollection', header=None, encoding='utf-8')

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5572 non-null   object
 1   1       5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None


In [4]:
print(df.head())

      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [5]:
# Variável para as classes para evitar ficar usando a notação df[0]
classes = df[0]

In [6]:
# imprimir a distribuição das classes
print(classes.value_counts())

ham     4825
spam     747
Name: 0, dtype: int64


ETAPA DE PRÉ-PROCESSAMENTO COM REGEX

In [7]:
import re

# padrão de re para encontrar números e caracteres especiais
pattern = re.compile(r'[\d@_!#$%^&*()<>?/\|}{~:]')

# selecionar a coluna de mensagens
text_messages = df[1]

# selecionar as 10 primeiras linhas
messages = text_messages[0:10]

# para cada linha na lista de mensagens
for line in messages:
  # criar lista de palavras dividindo a frase pelos espaços em branco
  words = line.split(' ')
  # para cada palavra na lista de palavras:
  for word in words:
    if re.search(pattern, word):
      print(word)

2
21st
2005.
87121
question(std
rate)T&C's
08452810075over18's
3
back!
still?
ok!
£1.50
(Oru
Vettam)'
*9
WINNER!!
£900
reward!
09061701461.
KL341.
12
11
more?
Free!
08002986030


In [8]:
# alterar todas as palavras para minúsculas
processed_lines = text_messages.str.lower()

# Lista de tuplas com os padrões e novas palavras
patterns = [
            # Substituir endereços de e-mail por 'emailaddress'
            (r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddress'),
            # Substituir URLS´s por 'webaddress'
            (r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress'),
            # Substituir simbolos monetários por 'moneysymb'
            (r'£|\$', 'moneysymb'),
            # Substituir números por 'numbr'
            (r'\d+(\.\d+)?', 'numbr'),
            # Remover pontuação
            (r'[^\w\d\s]', ''),
            # Substitua dois ou mais espaços em branco por um único espaço
            (r'\s+', ' '),
            # Remova os espaços em branco à esquerda e à direita
            (r'^\s+|\s+?$', '')
]

for pattern, newword in patterns:
  processed_lines = processed_lines.str.replace(pattern, newword)

In [9]:
print(processed_lines)

0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in numbr a wkly comp to win fa cup ...
3             u dun say so early hor u c already then say
4       nah i dont think he goes to usf he lives aroun...
                              ...                        
5567    this is the numbrnd time we have tried numbr c...
5568                  will ü b going to esplanade fr home
5569    pity was in mood for that soany other suggestions
5570    the guy did some bitching but i acted like id ...
5571                            rofl its true to its name
Name: 1, Length: 5572, dtype: object


In [10]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# criar um lista das stop words
stop_words = stopwords.words('english')

# inicializar uma instância de Porter Stemmer
ps = PorterStemmer()

# Para cada índice na lista
for i in range(len(processed_lines)):
  # criar tokens para cada linhas
  word_tokens = word_tokenize(processed_lines[i])
  # inicializar uma lista para receber as palavras filtradas
  filtered_sentence = []
  #para cada palavra da lista de tokens
  for word in word_tokens:
    if word not in stop_words:
      # cria um stem para a palavra
      stemmed_word = ps.stem(word)
      # adiciona na lista de palavras filtradas
      filtered_sentence.append(stemmed_word)

      # Junta as palavras da lista para substituir o antigo 
      # texto pelo novo texto processado:
      processed_lines[i] = ''.join(filtered_sentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Construção** **do** **modelo** **de** **predição**


In [11]:
#inicializar um lista de palavras:
all_words = []

# para cada linha da lista de linhas que foram processadas:
for line in processed_lines:
  #criar lista de palavras tokenizadas para cada linha:
  word_tokens = word_tokenize(line)
  # para cada palavra da lista de palavras:
  for word in word_tokens:
  # adiciona o token na bag onde estão todos os tokens
    all_words.append(word)

# Atribui a contagem de de frequência para cada token da lista:
all_words = nltk.FreqDist(all_words)

print(f'Total de palavras: {len(all_words)}')
print(f'Palavras mais comuns: {all_words.most_common(10)}')

#Usar as 1500 palavras mais comuns como modelo de features
# O método keys() da classe 'FreqDist' retorna somente
# os termos, que são as chaves da estrutura 'dict _ keys',
# por isso, foi usado list() para converter 'dict _ keys'
# para o tipo list
word_features = list(all_words.keys())[0:1500]
# Imprimir as 10 primeiras features (opcional)
print(f'Lista de características: {word_features[0:10]}')

Total de palavras: 5088
Palavras mais comuns: [('sorriillcalllater', 30), ('ok', 20), ('cantpickphonerightplsendmessag', 12), ('oki', 7), ('oklor', 5), ('sorriillcalllatermeet', 4), ('pleascallcustomservicrepresfreephonnumbrnumbrnumbrnumbramnumbrpmguarantemoneysymbnumbrcashmoneysymbnumbrprize', 4), ('wenurlovablbcumangriwidudnttakeseriouscozangrichildishntruewayshowdeepaffectcarenluvkettodamandanicedayda', 4), ('opinionnumbrnumbrjadanumbrkusruthinumbrlovablnumbrsilentnumbrsplcharactnumbrmaturnumbrstylishnumbrsimplplrepli', 4), ('privatnumbraccountstatementshownumbrunredeempointcallnumbridentificodenumbrexpirnumbrnumbrnumbr', 4)]
Lista de características: ['gojurongpointcraziavailbugingreatworldlaebuffetcinegotamorwat', 'oklarjokewifuoni', 'freeentrinumbrwklicompwinfacupfinaltktnumbrstmaynumbrtextfanumbrreceiventriquestionstdtxtratetcapplinumbrovernumbr', 'udunsayearlihorucalreadisay', 'nahdontthinkgoeusflivearoundthough', 'freemsgheydarlnumbrweekwordbackidlikefunstilltbokxxxstdchgsendm

Construção do modelo de predição

In [38]:
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder # biblioteca para converter valor categórico em discreto

# converter rótulos de classes para valores binários, 0 = ham e 1 = spam
encoder = LabelEncoder()
bin_labels = encoder.fit_transform(classes)

# Adicionar em uma lista de tuplas a mensagem e o rótulo da classe, de acordo com suas posições
# id 0 com id 0 , 1 com 1...
messages = list(zip(processed_lines, bin_labels))

#definir um valor de seed para reprodutibilidade
seed = 1
np.random.seed = stemmed_word

# embaralhar as mensagem para que a ordem não interfira no treinamento
np.random.shuffle(messages)

# inicializar uma lista geral para os vetores de features e os respectivos rótulos
featuresets = []

for text, label in messages:
 # Criar lista de palavras tokenizadas para cada linha
  word_tokens = word_tokenize(text)
 # Inicializar um set (conjunto) para as features
  features = {}
 # Para cada palavra existente na lista de tokens modelo
for word in word_features:
 # Se a palavra existir no texto, atribui True,
 # se não, False
  features[word] = (word in word_tokens)
 # Adiciona na lista geral como tupla de (vetor de
 #features e rótulos binarizados)
  featuresets.append((features, label))

In [ ]:
messages

In [39]:
# lib para dividir o dataset
from sklearn.model_selection import train_test_split

# dividir os dados em dois conjuntus; 75% para treinamento e 25% para teste
training, testing = train_test_split(
    featuresets, #dataset com as características e rótulos
    train_size = None,
    test_size = 0.25, 
    random_state =seed
)
# Imprimir o número de instâncias do conjunto de treinamento
print(len(training))

# Imprimir o número de instâncias do conjunto de teste
print(len(testing))

test_features, test_labels = zip(*testing)
train_features, train_labels = zip(*training)



1125
375


In [41]:
# Biblioteca do classificador multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
# lib para vetorizar dicionarios
from sklearn.feature_extraction import DictVectorizer
# instanciar
vect = DictVectorizer(sparse=False)

# Vetorizar os dados de treinamento e teste
train_features = vect.fit_transform(train_features)
test_features = vect.fit_transform(test_features)

# Instanciar o modelo Naive Bayes
model = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)

# Treinar o modelo com os dados de treinamento
model.fit(train_features, train_labels)

# Testar o modelo com os dados de teste
predict = model.predict(test_features)

AttributeError: ignored

In [42]:
# lib para gerar matriz de confusão
from sklearn.metrics import confusion_matrix

# gera matriz de confusão a partir dos resultados preditos
confus_matrix = confusion_matrix(test_labels, predict)

# cria um DF adicionando nomes de linhas e colunas
pd.DataFrame(
    confus_matrix,
    index = [['Real', 'Real'], ['spam', 'ham']],
    columns = [['Predito', 'Predito'], ['spam', 'ham']]
)

ValueError: ignored

In [43]:
print(test_features)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [24]:
train_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])